<!-- ---------------------------------------------------- -->
  <div class="col-sm-3 col-md-3 col-lg-3">
	<!-- logo -->  
    <div class="img-responsive">
      <img src="https://www.dropbox.com/s/220ncn0o5danuey/pandas-ipython-tutorials-hedaro.jpg?dl=1" title="Pandas Tutorial | Hedaro" alt="Pandas Tutorial | Hedaro">    
    </div>
	<!-- logo -->	
  </div>
<!-- ---------------------------------------------------- --> 
  <div class="col-sm-6 col-md-6 col-lg-6">
	<!-- Pandas Tutorial -->  
	  <center>
	    <br>
        <h1>Group By Practice</h1>
        <p>Let's practice Pandas using the <strong>Group By</strong> function and add some plots for fun.</p>
	  </center>	
    <!-- Pandas Tutorial -->	
  </div>
<!-- ---------------------------------------------------- -->

In [1]:
import pandas as pd
import sys

In [2]:
print('Python version ' + sys.version)
print('Pandas version ' + pd.__version__)

Python version 3.11.7 | packaged by Anaconda, Inc. | (main, Dec 15 2023, 18:05:47) [MSC v.1916 64 bit (AMD64)]
Pandas version 2.2.1


---

Here is the csv data if you want to follow along:
```
Date,Symbol,Volume
1/1/2013,A,0
1/2/2013,A,200
1/3/2013,A,1200
1/4/2013,A,1001
1/5/2013,A,1300
1/6/2013,A,1350
3/8/2013,B,500
3/9/2013,B,1150
3/10/2013,B,1180
3/11/2013,B,2000
1/5/2013,C,56600
1/6/2013,C,45000
1/7/2013,C,200
5/20/2013,E,1300
5/21/2013,E,1700
5/22/2013,E,900
5/23/2013,E,2100
5/24/2013,E,8000
5/25/2013,E,12000
5/26/2013,E,1900
5/27/2013,E,1000
5/28/2013,E,1900

```

In [3]:
raw = pd.read_csv('Test_9_17_Python.csv')
raw.head()

,Date,Symbol,Volume
0,1/1/2013,A,0
1,1/2/2013,A,200
2,1/3/2013,A,1200
3,1/4/2013,A,1001
4,1/5/2013,A,1300


In [4]:
df2 = raw.copy()

In [5]:
df2['Date'] = pd.to_datetime(df2['Date'])

In [6]:
df2.dtypes

Date      datetime64[ns]
Symbol            object
Volume             int64
dtype: object

In [7]:
pool = ['boy','girl']
pool = pool*int(len(df2)/2)
df2['Gender'] = pool
df = df2.copy()
df.head()

,Date,Symbol,Volume,Gender
0,2013-01-01,A,0,boy
1,2013-01-02,A,200,girl
2,2013-01-03,A,1200,boy
3,2013-01-04,A,1001,girl
4,2013-01-05,A,1300,boy


# Group one column  

[Click Here](http://www.hedaro.com/pandas-groupby) if you are unfamilar with the ***GroupBy*** function.

In [8]:
group = df.groupby('Symbol')

group.sum(numeric_only=True)

,Volume
Symbol,
A,5051
B,4830
C,101800
E,30800


When you use the groupby function in Pandas, you are grouping your data by one or more columns. These columns are called the "grouping columns" or "index columns".  

In our example we grouped by the column Symbol:  
> group = df.groupby('Symbol')  

The "Symbol" column becomes the grouping column.  

When you apply a function to this grouped DataFrame using apply, the function is currently applied to both the grouping columns (in this case, "Symbol") and the data columns (in this case, "Volume").  

However, in future versions of Pandas, the grouping columns will be excluded from the operation by default. This means that the function will only be applied to the data columns ("Volume" in this example), and not to the grouping column "Symbol".  

This change is being made to prevent unexpected behavior and to make the apply function more predictable. If you want to include the grouping columns in the operation, you will need to explicitly select them or pass include_groups=True.

In [9]:
def suma(group):
    return group.sum(numeric_only=True)

group.apply(suma, include_groups=False)

,Volume
Symbol,
A,5051
B,4830
C,101800
E,30800


In [10]:
group = df[['Symbol','Volume']].groupby('Symbol')

def suma(group):
    return group.sum(numeric_only=True)

df['addition'] = group.transform(suma)['Volume']
df.head(10)

,Date,Symbol,Volume,Gender,addition
0,2013-01-01,A,0,boy,5051
1,2013-01-02,A,200,girl,5051
2,2013-01-03,A,1200,boy,5051
3,2013-01-04,A,1001,girl,5051
4,2013-01-05,A,1300,boy,5051
5,2013-01-06,A,1350,girl,5051
6,2013-03-08,B,500,boy,4830
7,2013-03-09,B,1150,girl,4830
8,2013-03-10,B,1180,boy,4830
9,2013-03-11,B,2000,girl,4830


In [11]:
group = df.groupby('Symbol')

def test(group):
    mask1 = group.apply(lambda x: x.iloc[2]=='boy' and x.iloc[1]>1000, axis=1)
    return group[mask1]

group.apply(test, include_groups=False)

Date  Volume Gender  addition
Symbol                                       
A      2  2013-01-03    1200    boy      5051
       4  2013-01-05    1300    boy      5051
B      8  2013-03-10    1180    boy      4830
C      10 2013-01-05   56600    boy    101800
E      14 2013-05-21    1700    boy     30800
       16 2013-05-23    2100    boy     30800
       18 2013-05-25   12000    boy     30800

# Two columns

In [12]:
group = df.groupby(['Symbol', 'Gender'])

group.sum(numeric_only=True)

Volume  addition
Symbol Gender                  
A      boy       2500     15153
       girl      2551     15153
B      boy       1680      9660
       girl      3150      9660
C      boy      56800    203600
       girl     45000    101800
E      boy      16800    123200
       girl     14000    154000

In [13]:
def suma(group):
    return group['Volume'].sum(numeric_only=True)

group.apply(suma, include_groups=False)

Symbol  Gender
A       boy        2500
        girl       2551
B       boy        1680
        girl       3150
C       boy       56800
        girl      45000
E       boy       16800
        girl      14000
dtype: int64

In [14]:
group = df[['Symbol','Volume']].groupby('Symbol')

def suma(group):
    return group.sum(numeric_only=True)

df['addition'] = group.transform(suma)['Volume']
df.head()

,Date,Symbol,Volume,Gender,addition
0,2013-01-01,A,0,boy,5051
1,2013-01-02,A,200,girl,5051
2,2013-01-03,A,1200,boy,5051
3,2013-01-04,A,1001,girl,5051
4,2013-01-05,A,1300,boy,5051


In [15]:
group = df.groupby('Symbol')

def test(group):
    mask1 = group.apply(lambda x: x.iloc[1]>1000, axis=1)
    return group[mask1]

group.apply(test)

Date  Volume Gender  addition
Symbol                                       
A      2  2013-01-03    1200    boy      5051
       3  2013-01-04    1001   girl      5051
       4  2013-01-05    1300    boy      5051
       5  2013-01-06    1350   girl      5051
B      7  2013-03-09    1150   girl      4830
       8  2013-03-10    1180    boy      4830
       9  2013-03-11    2000   girl      4830
C      10 2013-01-05   56600    boy    101800
       11 2013-01-06   45000   girl    101800
E      13 2013-05-20    1300   girl     30800
       14 2013-05-21    1700    boy     30800
       16 2013-05-23    2100    boy     30800
       17 2013-05-24    8000   girl     30800
       18 2013-05-25   12000    boy     30800
       19 2013-05-26    1900   girl     30800
       21 2013-05-28    1900   girl     30800

<p class="text-muted">This tutorial was created by <a href="http://www.hedaro.com" target="_blank"><strong>HEDARO</strong></a></p>